# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [100]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [101]:
print('Few rows of portfolio dataset are')
portfolio.head()

Few rows of portfolio dataset are


,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [102]:
print('Few rows of profile dataset are')
profile.head()

Few rows of profile dataset are


,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [103]:
print('Few rows of transcript dataset are')
transcript.head()

Few rows of transcript dataset are


,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [104]:
print('Num of different offers are : ' + str(portfolio.shape[0]))
print('Num of customers are        : ' + str(profile.shape[0]))
print('Num of events are           : ' + str(transcript.shape[0]))

Num of different offers are : 10
Num of customers are        : 17000
Num of events are           : 306534


Seems there are no rows with Null values in portfolio and transcript datasets.

### Portfolio Dataset

In [105]:
portfolio.dtypes

channels      object
difficulty     int64
duration       int64
id            object
offer_type    object
reward         int64
dtype: object

In [106]:
portfolio["offer_type"].value_counts()

bogo             4
discount         4
informational    2
Name: offer_type, dtype: int64

In [107]:
to_numerical_offer_types = {"offer_type" : {"discount": 1, "bogo": 2, "informational" : 3}}
portfolio.replace(to_numerical_offer_types, inplace=True)
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,2,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,2,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,3,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,2,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5


In [108]:
# Encoding email, web, social and mobile in new columns
# One hot encoding event into new columns - offer received, offer viewed, offer completed, transaction
portfolio['offer received'] = [ 1 if event=='offer received' else 0 for event in portfolio['channels'] ]
portfolio['offer viewed'] = [ 1 if event=='offer viewed' else 0 for event in portfolio['channels'] ]
portfolio['offer completed'] = [ 1 if event=='offer completed' else 0 for event in portfolio['channels'] ]
portfolio['transaction'] = [ 1 if event=='transaction' else 0 for event in portfolio['channels'] ]
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward,offer received,offer viewed,offer completed,transaction
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,2,10,0,0,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,2,10,0,0,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,3,0,0,0,0,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,2,5,0,0,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,0,0,0,0


In [109]:
# dropping channels column
portfolio = portfolio.drop(['channels'], axis=1)
portfolio.head()

,difficulty,duration,id,offer_type,reward,offer received,offer viewed,offer completed,transaction
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,2,10,0,0,0,0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,2,10,0,0,0,0
2,0,4,3f207df678b143eea3cee63160fa8bed,3,0,0,0,0,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,2,5,0,0,0,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,5,0,0,0,0


### Profile Dataset

In [110]:
# Renaming person column to person id
profile.rename(columns={'id':'person_id'}, inplace=True)
profile.head()

,age,became_member_on,gender,person_id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [111]:
# Dropping rows with null values
print('Num of rows in profile before dropping rows with null values    : '+str(profile.shape[0]))
profile = profile.dropna()
print('Num of rows in profile before dropping rows with null values    : '+str(profile.shape[0]))

Num of rows in profile before dropping rows with null values    : 17000
Num of rows in profile before dropping rows with null values    : 14825


In [112]:
profile.head()

,age,became_member_on,gender,person_id,income
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
12,58,20171111,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0


In [113]:
profile["gender"].value_counts()

M    8484
F    6129
O     212
Name: gender, dtype: int64

In [114]:
# Converting gender from categorical to numerical
to_numerical_gender = {"gender" : {"M": 1, "F": 0}}
profile.replace(to_numerical_gender, inplace=True)
profile.head()

,age,became_member_on,gender,person_id,income
1,55,20170715,0,0610b486422d4921ae7d2bf64640c50b,112000.0
3,75,20170509,0,78afa995795e4d85b5d9ceeca43f5fef,100000.0
5,68,20180426,1,e2127556f4f64592b11af22de27a7932,70000.0
8,65,20180209,1,389bc3fa690240e798340f5a15918d5c,53000.0
12,58,20171111,1,2eeac8d8feae4a8cad5a6af0499a211d,51000.0


In [115]:
profile['year'] = profile['became_member_on'].apply(lambda x: str(x)[:4])
profile.head()

,age,became_member_on,gender,person_id,income,year
1,55,20170715,0,0610b486422d4921ae7d2bf64640c50b,112000.0,2017
3,75,20170509,0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017
5,68,20180426,1,e2127556f4f64592b11af22de27a7932,70000.0,2018
8,65,20180209,1,389bc3fa690240e798340f5a15918d5c,53000.0,2018
12,58,20171111,1,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017


In [116]:
profile['year'].value_counts()

2017    5599
2018    3669
2016    3024
2015    1597
2014     662
2013     274
Name: year, dtype: int64

In [117]:
profile = profile.drop(['became_member_on'], axis=1)

In [118]:
one_hot_encode_year = pd.get_dummies(profile['year'], prefix='year')
one_hot_encode_year.head()

,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
1,0,0,0,0,1,0
3,0,0,0,0,1,0
5,0,0,0,0,0,1
8,0,0,0,0,0,1
12,0,0,0,0,1,0


In [120]:
profile = pd.concat([profile, one_hot_encode_year], axis=1)
profile.shape[0]

14825

In [122]:
profile = profile.drop(['year'], axis=1)
profile.head()

,age,gender,person_id,income,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
1,55,0,0610b486422d4921ae7d2bf64640c50b,112000.0,0,0,0,0,1,0
3,75,0,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0,0,0,0,1,0
5,68,1,e2127556f4f64592b11af22de27a7932,70000.0,0,0,0,0,0,1
8,65,1,389bc3fa690240e798340f5a15918d5c,53000.0,0,0,0,0,0,1
12,58,1,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,0,0,0,0,1,0


### Transcript Dataset

In [59]:
# Renaming person column to person id
transcript.rename(columns={'person':'person_id'}, inplace=True)
transcript.head()

,event,person_id,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [61]:
# Removing customer id's that are not in the profile DataFrame
print('Num rows in transcript dataframe before removing customers for which we dont have profile is : ' + str(transcript.shape[0]))
transcript = transcript[transcript['person_id'].isin(profile['person_id'])]
print('Num rows in transcript dataframe after removing customers for which we dont have profile is  : ' + str(transcript.shape[0]))

Num rows in transcript dataframe before removing customers for which we dont have profile is : 272762
Num rows in transcript dataframe after removing customers for which we dont have profile is  : 272762


In [62]:
transcript['event'].value_counts()

transaction        123957
offer received      66501
offer viewed        49860
offer completed     32444
Name: event, dtype: int64

In [17]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [18]:
# One hot encoding event into new columns - offer received, offer viewed, offer completed, transaction
transcript['offer_received'] = [ 1 if event=='offer received' else 0 for event in transcript['event'] ]
transcript['offer_viewed'] = [ 1 if event=='offer viewed' else 0 for event in transcript['event'] ]
transcript['offer_completed'] = [ 1 if event=='offer completed' else 0 for event in transcript['event'] ]
transcript['transaction'] = [ 1 if event=='transaction' else 0 for event in transcript['event'] ]
transcript.head()

,event,person,time,value,offer_received,offer_viewed,offer_completed,transaction
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1,0,0,0
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},1,0,0,0
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},1,0,0,0
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1,0,0,0
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1,0,0,0


In [19]:
# dropping event column
transcript = transcript.drop(['event'], axis=1)
transcript.head()

,person,time,value,offer_received,offer_viewed,offer_completed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1,0,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},1,0,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},1,0,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1,0,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1,0,0,0


In [20]:
transcript['value'].value_counts()

{'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}                  14983
{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}                  14924
{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}                  14891
{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}                  14835
{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}                  14374
{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}                  14305
{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}                  11848
{'offer id': '3f207df678b143eea3cee63160fa8bed'}                  11761
{'offer id': '2906b810c7d4411798c6938adc9daaa5'}                  11750
{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}                  10331
{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4', 'reward': 2}      5317
{'offer_id': '2298d6c36e964ae4a3e7e9706d1fb8c2', 'reward': 3}      5156
{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9', 'reward': 5}      4354
{'offer_id': 'f19421c1d4aa40978ebb69ca19b0e20d', 'reward': 5}   

In [21]:
transcript['offer_id'] = [x['offer id'] if 'offer id' in x.keys() else 0 for x in transcript['value']]
transcript['amount']   = [x['amount'] if 'amount' in x.keys() else 0 for x in transcript['value']]
transcript['reward']   = [x['reward'] if 'reward' in x.keys() else 0 for x in transcript['value']]
transcript.head()

,person,time,value,offer_received,offer_viewed,offer_completed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},1,0,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},1,0,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1,0,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},1,0,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0


In [22]:
# dropping channels column
transcript = transcript.drop(['value'], axis=1)
transcript.head()

,person,time,offer_received,offer_viewed,offer_completed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,1,0,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0
1,a03223e636434f42ac4c3df47e8bac43,0,1,0,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.0,0
2,e2127556f4f64592b11af22de27a7932,0,1,0,0,0,2906b810c7d4411798c6938adc9daaa5,0.0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,1,0,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0
4,68617ca6246f4fbc85e91a2a49552598,0,1,0,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.0,0


In [23]:
transcript.shape

(306534, 9)

In [33]:
transaction_data = transcript[transcript.transaction == 1]
transaction_data.shape

(138953, 9)

In [34]:
transaction_data= transaction_data.drop(['offer_id', 'offer_received', 'offer_viewed', 'offer_completed', 'transaction', 'reward'], axis=1)
transaction_data.head()

,person_id,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


In [35]:
offer_data = transcript[transcript.transaction == 0]
offer_data.shape

(167581, 9)

In [36]:
offer_data = offer_data.drop(['transaction', 'amount'], axis=1)
offer_data.head()

,person_id,time,offer_received,offer_viewed,offer_completed,offer_id,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0
1,a03223e636434f42ac4c3df47e8bac43,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0
2,e2127556f4f64592b11af22de27a7932,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0
4,68617ca6246f4fbc85e91a2a49552598,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0
